In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [2]:
system_prompt = "あなたは日本語で回答するアシスタントです。"

prompt = """
【問題】
X、Y、Zが同じ絵を見てて、次のように発言した。

X　この絵には虎が描かれている。
Y　この絵には少なくとも虎か龍が描かれている。
Z　この絵には龍が描かれている。

全員が本当のことを言っているとは限らない。そこで、以下の推論がなされた。次のうち正しいものを一つ選びなさい。

＜選択肢＞
A.Yが正しければXは必ず正しい
B.Zが正しければYは必ず正しい
C.Xが正しければZは必ず正しい
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True #ハードスイッチ
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

try:
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, let's try to figure out this logic problem. So, there are three people, X, Y, and Z, who saw the same picture and made statements. The problem says that not all of them are necessarily telling the truth. Then there are three options (A, B, C) that we need to evaluate as correct or not.

First, let me restate their statements to make sure I understand them correctly.

X says: "This picture has a tiger drawn in it." So, X's statement is that there is at least one tiger in the picture.

Y says: "This picture has at least a tiger or a dragon drawn in it." So Y's statement is that the picture contains either a tiger or a dragon, or both. In logical terms, that's "Tiger ∨ Dragon".

Z says: "This picture has a dragon drawn in it." So Z is asserting that there's at least one dragon in the picture.

Now, the question is about which of the options A, B, or C is correct. Let me go through each option one by one.

Option A: "If Y is telling the truth, then X must be

In [3]:
system_prompt = "あなたは日本語で回答するアシスタントです。"

#ソフトスイッチ
prompt = """
虎は何科ですか？/no_think
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True #ハードスイッチ
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

try:
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>

</think>
content: 虎は「ネコ科（Felidae）」に属します。  
虎はネコ科の「虎属（Panthera）」に分類され、他のネコ科の動物（例えば、トラ、ユキネコ、イエネコなど）と親戚関係があります。


In [4]:
!ollama pull qwen3:8b

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling a3de86cd1c13:   0% ▕                  ▏ 6.8 MB/5.2 GB                  pulling manifest 
pulling a3de86cd1c13:   0% ▕                  ▏  15 MB/5.2 GB                  pulling manifest 
pulling a3de86cd1c13:   1% ▕                  ▏  30 MB/5.2 GB                  pulling manifest 
pulling a3de86cd1c13:   1% ▕                  ▏  44 MB/5.2 GB                  pulling manifest 
pulling a3de86cd1c13:   1% ▕                  ▏  53 MB/5.2 GB                  pu

In [5]:
import os
from qwen_agent.agents import Assistant

# ステップ1: 使用するLLMを設定
llm_cfg = {
    'model': 'qwen3:8b',
    'model_server': 'http://localhost:11434/v1',
    'api_key': 'ollama',
    'generate_cfg': {
        'top_p': 0.8
    }
}

In [6]:
from qwen_agent.agents import Assistant

# Define LLM
llm_cfg = {
    'model':'qwen3:8b',

    # Use the endpoint provided by Alibaba Model Studio:
    # 'model_type': 'qwen_dashscope',
    # 'api_key': os.getenv('DASHSCOPE_API_KEY'),

    # Use a custom endpoint compatible with OpenAI API:
    'model_server': 'http://localhost:11434/v1',
    'api_key': 'ollama',

    # Other parameters:
    # 'generate_cfg': {
    #         # Add: When the response content is `<think>this is the thought</think>this is the answer;
    #         # Do not add: When the response has been separated by reasoning_content and content.
    #         'thought_in_content': True,
    #     },
}

tools = ['code_interpreter']

bot = Assistant(llm=llm_cfg, function_list=tools)

messages = [{'role': 'user', 'content': 'print(123**2)'}]
for responses in bot.run(messages=messages):
    pass
print(responses)

2025-06-23 14:17:31,046 - code_interpreter.py - 187 - INFO - INFO: kernel process's PID = 392
2025-06-23 14:17:33,173 - code_interpreter.py - 119 - INFO - stdout:

```
Exception reporting mode: Minimal

```


[{'role': 'assistant', 'content': '<think>\nOkay, the user wants to print 123 squared. Let me think about how to approach this. The code interpreter function is available, so I can use that to run the Python code. The task is straightforward: calculate 123 raised to the power of 2 and print the result.\n\nFirst, I need to write the Python code. The expression is 123 ** 2. Using the code interpreter function, I can execute this code and return the output to the user. Let me make sure there are no syntax errors. The code should be as simple as print(123 ** 2). \n\nWait, the user might just want the code executed and the result shown. Since the code interpreter handles the execution, I should structure the tool call correctly. The function requires the code parameter, so I\'ll format the code accordingly. No other parameters are needed. Let me double-check the function signature to ensure I\'m using the right parameters. Yes, the code parameter is a string and required. \n\nAlright, I\'ll